In [1]:
class Genome:
    def __init__(self, hit_probability, ace_is_one_probability):
        self.hit_probability = hit_probability
        self.ace_is_one_probability = ace_is_one_probability
        self.fitness = 0
    
    def __str__(self):
        res = "Hit: "
        for probability in self.hit_probability:
            res += str(probability)
            res += str(" ")
            
        res += "\nCount ace as 1: "
        for probability in self.ace_is_one_probability:
            res += str(probability)
            res += str(" ")
            
        return res

In [2]:
import random
from tqdm import tnrange, tqdm_notebook

class GeneticAlgorithm:
    def __init__(self, target_score, target_win_rate):
        self.target_fitness = target_score * target_win_rate
        self.fitness_rounds = 1000
        self.generation_size = 1000
        self.reproducers_size = 200
        self.max_iterations = 1000
        self.mutation_rate = 0.1
        self.tournament_size = 10
        self.selection_type = 'roulette' #can be 'tournament' or 'roulette'
        
    ''' Calculates the sum of cards in hand, picking whether to count aces as 1 or 11
        based on a genetically passed down probability.'''
    def calculate_player_preferred_sum(self, sum_without_aces, ace_count, ace_is_one_probability):
        total_sum = sum_without_aces
        while(ace_count>0):
            probability = random.randint(0,100)
            if(total_sum+11>21):
                total_sum += ace_count
                break;
            elif(probability<ace_is_one_probability[total_sum]):
                ace_count -= 1
                total_sum += 11
            else:
                ace_count -= 1
                total_sum += 1
                
        return total_sum
     
    ''' Simulates one game of Blackjack, returns a tuple (Boolean, int)
        containing whether it's a win and what the score was.'''
    def get_game_results(self, genome):
        hit_probability = genome.hit_probability
        ace_is_one_probability = genome.ace_is_one_probability
        
        ace_count = 0
        sum_without_aces = 0
        player_preferred_sum = 0
        win = False
        
        #deal starting two cards
        for i in range(2):
            card = random.randint(1,10)
            if(card == 1):
                ace_count += 1
            else:
                sum_without_aces += card
            
        while sum_without_aces+ace_count<=21:
            player_preferred_sum = self.calculate_player_preferred_sum(sum_without_aces, ace_count, ace_is_one_probability)
            if player_preferred_sum == 21:
                win = True
                break
            elif player_preferred_sum > 21:
                win = False
                break
            hit = random.randint(1,100)
            
            if(hit<hit_probability[player_preferred_sum]):
                card = random.randint(1,10)
                if(card == 1):
                    ace_count += 1
                else:
                    sum_without_aces += card
            else:
                win = True
                break
        
        return (win, player_preferred_sum)
        
    ''' Calculates the fitness of a genome, based on its average number
        of wins and its average score when winning.'''
    def calculate_fitness(self, genome):
        wins = 0
        total = 0
        for i in range(self.fitness_rounds):
            game = self.get_game_results(genome)
            if game[0]:
                wins += 1
                total += game[1]
        win_rate = wins / self.fitness_rounds
        average_score = total / self.fitness_rounds
        return int(win_rate*average_score)
    
    ''' Initializies the first population randomly. '''
    def initial_population(self):
        init_population = []
        
        for i in range(self.generation_size):
            hit_probability = []
            ace_is_one_probability = []
            for j in range(21): # note that 0 and 1 are unimportant
                hit_probability.append(random.randint(1,100))
                ace_is_one_probability.append(random.randint(1,100))
            new_speciman = Genome(hit_probability, ace_is_one_probability)
            new_speciman.fitness = self.calculate_fitness(new_speciman)
            init_population.append(new_speciman)
            
        return init_population
    
    ''' Selects genomes for reproduction. '''
    def selection(self, genomes):
        selected = []
        
        for i in range(self.reproducers_size):
            if self.selection_type == 'roulette':
                selected.append(self.roulette_selection(genomes))
            elif self.selection_type == 'tournament':
                selected.append(self.tournament_selection(genomes))
          
        return selected
    
    ''' Slects a genome with the probability equal to its fitness divided
        by the overall fitness of the population '''
    def roulette_selection(self, population):
        total_fitness = sum([genome.fitness for genome in population])
        
        selected_value = random.randint(0, total_fitness)
        
        current_sum = 0
        for genome in population:
            current_sum += genome.fitness
            if current_sum > selected_value:
                return genome
        return random.choice(population)
         
    ''' Selects the best speciman from a tournament of random 
        self.tournament_size genomes. '''
    def tournament_selection(self, population):
        selected = random.sample(population, self.tournament_size)
        winner = max(selected, key = lambda x: x.fitness)
        return winner
    
    ''' Mutates a genome with the probability of self.mutation_rate. '''
    def mutate(self, hit_probability, ace_is_one_probability):
        mutation = random.random()
        if mutation > self.mutation_rate:
            hit = random.randint(0,20)
            hit_probability[hit] = random.randint(1,100)
        
        mutation = random.random()
        if mutation > self.mutation_rate:
            ace = random.randint(0,20)
            ace_is_one_probability[ace] = random.randint(1,100)
            
        return (hit_probability, ace_is_one_probability)
    
    def create_generation(self, population):
        generation = []
        generation_size = 0
        
        while generation_size < self.generation_size:
            
            [parent1, parent2] = random.sample(population, 2)
            
            child1_code, child2_code = self.crossover(parent1, parent2)
            
            child1_code = self.mutate(child1_code.hit_probability, child1_code.ace_is_one_probability)
            child2_code = self.mutate(child2_code.hit_probability, child2_code.ace_is_one_probability)
            
            child1 = Genome(child1_code[0], child1_code[1])
            child1.fitness = self.calculate_fitness(child1)
            child2 = Genome(child2_code[0], child2_code[1])
            child2.fitness = self.calculate_fitness(child2)
            
            generation.append(child1)
            generation.append(child2)
            
            generation_size += 2
            
        return generation
    
    
    
    def crossover(self, parent1, parent2):
        child1 = Genome(parent1.hit_probability, parent1.ace_is_one_probability)
        child2 = Genome(parent2.hit_probability, parent2.ace_is_one_probability)
        
        break_point = random.randrange(1, 21)
        
        child1.hit_probability = parent1.hit_probability[:break_point] + parent2.hit_probability[break_point:]
        child2.hit_probability = parent2.hit_probability[:break_point] + parent1.hit_probability[break_point:]
        
        break_point = random.randrange(1, 21)
        child1.ace_is_one_probability = parent1.ace_is_one_probability[:break_point] + parent2.ace_is_one_probability[break_point:]
        child2.ace_is_one_probability = parent2.ace_is_one_probability[:break_point] + parent1.ace_is_one_probability[break_point:]
        
        child1.fitness = self.calculate_fitness(child1)
        child2.fitness = self.calculate_fitness(child2)
        
        return (child1, child2)
    
    
    
    def optimize(self):
        population = self.initial_population()
        
        for i in tnrange(self.max_iterations, desc='Generation'):
            selected = self.selection(population)
            population = self.create_generation(selected)
            global_best_genome = max(population, key=lambda x: x.fitness)
            #print(global_best_genome)
            if global_best_genome.fitness >= self.target_fitness:
                break
                
        return global_best_genome

In [3]:
genetic_algorithm = GeneticAlgorithm(21, 0.8)
result = genetic_algorithm.optimize()

print('Result:\n{}'.format(result))

KeyboardInterrupt: 

In [ ]:
def compare_fitness(target_score, target_win_rate, no_of_rounds, printOutput):
    genetic_algorithm = GeneticAlgorithm(target_score, target_win_rate)
    
    alice = genetic_algorithm.optimize()
    if printOutput:
        print('Result:\n{}'.format(alice))
    bob   = genetic_algorithm.optimize()
    if printOutput:
        print('Result:\n{}'.format(bob))
    
    alice_wins = 0
    bob_wins   = 0
    
    if printOutput:
        for i in tnrange(no_of_rounds,desc="Simulating Games"):
            game_alice = genetic_algorithm.get_game_results(alice)
            game_bob   = genetic_algorithm.get_game_results(bob)

            if(game_alice[0] and game_alice[1] > game_bob[1]):
                alice_wins += 1
            elif(game_bob[0] and game_bob[1] > game_alice[1]):
                bob_wins += 1
            else:
                alice_wins += 0.5
                bob_wins += 0.5
    else:
        for i in range(no_of_rounds):
            game_alice = genetic_algorithm.get_game_results(alice)
            game_bob   = genetic_algorithm.get_game_results(bob)

            if(game_alice[0] and game_alice[1] > game_bob[1]):
                alice_wins += 1
            elif(game_bob[0] and game_bob[1] > game_alice[1]):
                bob_wins += 1
            else:
                alice_wins += 0.5
                bob_wins += 0.5

    alice_fitness = alice_wins / no_of_rounds
    bob_fitness   = bob_wins / no_of_rounds

        
    if printOutput:
        print("alice: {}\nbob: {}\n".format(alice_fitness, bob_fitness))
    
    return alice_fitness, bob_fitness

In [ ]:
compare_fitness(21,0.8,1000,True)

In [ ]:
def compare_against_random(target_score, target_win_rate, no_of_optimizations, no_of_rounds):
    random_fitness  = 0
    evolved_fitness = 0
    
    for i in tnrange(no_of_optimizations,desc="Simulating games"):
        hit_probability = []
        ace_is_one_probability = []
        for j in range(21): # note that 0 and 1 are unimportant
            hit_probability.append(random.randint(1,100))
            ace_is_one_probability.append(random.randint(1,100))
        random_genome = Genome(hit_probability, ace_is_one_probability)

        genetic_algorithm = GeneticAlgorithm(target_score, target_win_rate)
        evolved_genome = genetic_algorithm.optimize()

        random_wins = 0
        evolved_wins = 0
        for i in range(no_of_rounds):
            game_random  = genetic_algorithm.get_game_results(random_genome)
            game_evolved = genetic_algorithm.get_game_results(evolved_genome)

            if(game_random[0] and game_random[1] > game_evolved[1]):
                random_wins += 1
            elif(game_evolved[0] and game_evolved[1] > game_random[1]):
                evolved_wins += 1
            else:
                random_wins += 0.5
                evolved_wins += 0.5

        random_fitness  += random_wins / no_of_rounds
        evolved_fitness += evolved_wins / no_of_rounds
        
    random_fitness  /= no_of_optimizations
    evolved_fitness /= no_of_optimizations
    
    print("random: {}\nevolved: {}\n".format(random_fitness, evolved_fitness))
    
    return random_fitness, evolved_fitness

In [ ]:
random_fitness, evolved_fitness = compare_against_random(21, 0.8, 12, 1000)